# 시퀀스 배열로 다루는 순환 신경망(RNN)

### 1. LSTM을 이용한 로이터 뉴스 카테고리 분류하기

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import reuters       # 로이터 뉴스 데이터셋 불러오기
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
help(reuters)

Help on package keras.api._v2.keras.datasets.reuters in keras.api._v2.keras.datasets:

NAME
    keras.api._v2.keras.datasets.reuters - Reuters topic classification dataset.

PACKAGE CONTENTS


FILE
    c:\programdata\anaconda3\envs\nlpdl\lib\site-packages\keras\api\_v2\keras\datasets\reuters\__init__.py




In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# 데이터를 불러와 학습셋, 테스트셋으로 나눕니다.
(X_train, y_train), (X_test, y_test) = reuters.load_data(
    num_words=1000,
    # 임베딩 층을 설정할 때 사용하는 매개변수
    # 단어 사전의 크기 지정
    
    # 자주 등장하는 상위 1000개의 단어만을 사용하는 경우 유용하다
    # 단어 사전의 크기를 제한하여 메모리 사용을 줄이고 계산 효율성을 높인다
    test_split=0.2)

In [5]:
# 데이터를 확인해 보겠습니다.
category = np.max(y_train) + 1
print(category, '카테고리')
print(len(X_train), '학습용 뉴스 기사')
print(len(X_test), '테스트용 뉴스 기사')
print(X_train[0])

46 카테고리
8982 학습용 뉴스 기사
2246 테스트용 뉴스 기사
[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 2, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 2, 2, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 2, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]


In [6]:
X_train[0]

[1,
 2,
 2,
 8,
 43,
 10,
 447,
 5,
 25,
 207,
 270,
 5,
 2,
 111,
 16,
 369,
 186,
 90,
 67,
 7,
 89,
 5,
 19,
 102,
 6,
 19,
 124,
 15,
 90,
 67,
 84,
 22,
 482,
 26,
 7,
 48,
 4,
 49,
 8,
 864,
 39,
 209,
 154,
 6,
 151,
 6,
 83,
 11,
 15,
 22,
 155,
 11,
 15,
 7,
 48,
 9,
 2,
 2,
 504,
 6,
 258,
 6,
 272,
 11,
 15,
 22,
 134,
 44,
 11,
 15,
 16,
 8,
 197,
 2,
 90,
 67,
 52,
 29,
 209,
 30,
 32,
 132,
 6,
 109,
 15,
 17,
 12]

In [7]:
X_test.shape

(2246,)

In [8]:
# 단어의 수를 맞추어 줍니다. 
X_train = sequence.pad_sequences(X_train, maxlen=100)
X_test = sequence.pad_sequences(X_test, maxlen=100)

# 원-핫 인코딩 처리를 합니다.
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [9]:
# 모델의 구조를 설정합니다.
model = Sequential()
model.add(Embedding(1000, 100))
# (입력 단어의 사전 크기, 각 단어가 매핑될 임베딩 벡터의 차원)
model.add(LSTM(100, activation='tanh'))
# LSTM 층은 100개의 상태를 갖는 메모리 셀을 가지고,시퀀스 정보를 처리합니다
# 하이퍼볼릭 탄젠트
model.add(Dense(46, activation='softmax'))

In [10]:
# 모델의 실행 옵션을 정합니다.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 학습의 조기 중단을 설정합니다.
early_stopping_callback = EarlyStopping(monitor='val_loss',
                                        patience=5)

# 모델을 실행합니다.
history = model.fit(X_train, y_train, batch_size=20,
                    epochs=200, validation_data=(X_test, y_test),
                    callbacks=[early_stopping_callback])

# 테스트 정확도를 출력합니다.
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, y_test)[1]))

Epoch 1/200
450/450 [==============================] - 22s 46ms/step - loss: 2.1959 - accuracy: 0.4336 - val_loss: 1.8439 - val_accuracy: 0.5356
Epoch 2/200
450/450 [==============================] - 19s 41ms/step - loss: 1.7987 - accuracy: 0.5355 - val_loss: 1.7243 - val_accuracy: 0.5744
Epoch 3/200
450/450 [==============================] - 20s 43ms/step - loss: 1.6237 - accuracy: 0.5851 - val_loss: 1.6028 - val_accuracy: 0.5899
Epoch 4/200
450/450 [==============================] - 19s 42ms/step - loss: 1.4645 - accuracy: 0.6317 - val_loss: 1.4696 - val_accuracy: 0.6327
Epoch 5/200
450/450 [==============================] - 19s 41ms/step - loss: 1.2779 - accuracy: 0.6847 - val_loss: 1.3249 - val_accuracy: 0.6594
Epoch 6/200
450/450 [==============================] - 18s 41ms/step - loss: 1.1504 - accuracy: 0.7143 - val_loss: 1.2401 - val_accuracy: 0.6861
Epoch 7/200
450/450 [==============================] - 19s 42ms/step - loss: 1.0539 - accuracy: 0.7308 - val_loss: 1.1912 - val_ac